<a href="https://colab.research.google.com/github/KankshiDhar/portfolio/blob/master/JLTextMining(Step2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install num2words
!pip install plotly

In [10]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
from num2words import num2words

import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle
import re
import math
import string

from tkinter import filedialog
from tkinter import *

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

In [11]:
### Select jlByFunction.csv file from your working directory ###

from google.colab import files
uploaded = files.upload()

Saving jlByFunction.csv to jlByFunction.csv


In [12]:
jobs = pd.read_csv('jlByFunction.csv')
jobs.head()

,Unnamed: 0,Company Name,Industry,Job Description,Job_Function,Job Posting Date,Job Status,Job Title,Job Type,Location,Student Groups,US Work Auth Requirement,Unique ID,month,day,year
0,0,"BDO USA, LLP",Other,"BDO’s summer leadership program, Pathway to Su...",Other,2/16/2011 0:00,Approved,Pathway To Success Summer Leadership Program S...,Temporary/Seasonal,NaN,NaN,Position requires permanent US Work Authorization,15111.0,2,16,2011
1,1,McKinsey & Company,Consulting - Other,\rSpecialists are dedicated to working with cl...,Marketing/Business/Sales Management,7/20/2011 13:10,Approved,Marketing Analytic Specialists,Full-Time,"NORTH AMERICA, NORTH AMERICA - USA, NORTH AMER...",Alumni - MBAs,Position requires permanent US Work Authorization,1937461.0,7,20,2011
2,2,First Annapolis Consulting,Other,"First Annapolis Consulting, Inc., offers a car...",Other,2/22/2011 0:00,Approved,Consultant,Full-Time,NaN,NaN,Position requires permanent US Work Authorization,14200.0,2,22,2011
3,3,"JK Group, Inc.",Consulting - Technology,PRIMARY RESPONSIBILITIES Manage the company’s ...,Marketing/Business/Sales Management,7/20/2011 11:19,Approved,VP Operations,Full-Time,NORTH AMERICA - USA - USA - NORTHEAST - New Je...,"MBAs Seeking Full-Time, Alumni - MBAs",Position requires permanent US Work Authorization,1937418.0,7,20,2011
4,4,EPIC Academy Charter High School,Education,\rEPIC Academy’s early work as a startup organ...,Financial Services,7/20/2011 7:31,Approved,Chief Finance and Operations Officer,Full-Time,NORTH AMERICA - USA - USA - MIDWEST - Illinois...,"MBAs Seeking Full-Time, Alumni - MBAs",Position requires permanent US Work Authorization,1937376.0,7,20,2011


In [13]:
### Select the skillwords.csv from your working directory ###
### Select jlByFunction.csv file from your working directory ###

from google.colab import files
uploaded = files.upload()

Saving TopSkills.csv to TopSkills.csv


In [14]:
## hard skills/tool/concept knowledge required ##
s = pd.read_csv('TopSkills.csv')
s['Skills'] = s['Skills'].str.strip()
skillset = s['Skills'].unique()

In [15]:
data = jobs['Job Description'].tolist()

In [16]:
def convert_lower_case(data):
    return np.char.lower(data)

def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
    return new_text

def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data

def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

def stemming(data):
    stemmer= PorterStemmer()
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return new_text

def convert_numbers(data):
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        try:
            w = num2words(int(w))
        except:
            a = 0
        new_text = new_text + " " + w
    new_text = np.char.replace(new_text, "-", " ")
    return new_text

In [17]:
def preprocess(data):
    data = convert_lower_case(data)
    data = remove_punctuation(data) #remove comma seperately
    data = remove_apostrophe(data)
    data = remove_stop_words(data)
    data = convert_numbers(data)
    data = stemming(data)
    data = remove_punctuation(data)
    data = convert_numbers(data)
    data = stemming(data) #needed again as we need to stem the words
    data = remove_punctuation(data) #needed again as num2word is giving few hypens and commas fourty-one
    data = remove_stop_words(data) #needed again as num2word is giving stop words 101 - one hundred and one
    return data

In [19]:
import nltk
import gensim
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [20]:
for i in range(len(data)):
    data[i] = preprocess(data[i])

In [41]:
# This saves the list data as a csv named "JDPreprocessedData.csv" to save processing time

## To resume this process from here, run the next code cell
datadf = pd.DataFrame(data, columns=['JD_Token'])
datadf.to_csv('JDPreprocessedData.csv', index=None, header = True)
files.download('JDPreprocessedData.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [39]:
# Run this cell to resume process from this point.
# Select file JDPreprocessedData.csv here

from google.colab import files
uploaded = files.upload()
datadf = pd.read_csv('JDPreprocessedData.csv')
data = list(datadf['JD_Token'])

 first annapoli consult inc offer career opportun except individu motiv challeng help major corpor address real world problem first annapoli manag consult merger acquisit firm primari focu financ servic industri princip specialti payment relat product servic credit debit prepaid product financ transact process commerci payment emerg payment product compri one third us bank revenu first annapoli largest independ profess servic compani focu industri segment client includ lead compani financ servic retail technolog consum good industri primarili north america increasingli europ south america consult work direct firm partner princip senior manag wide varieti project quickli assum respon manag key task project supervi work research analysi staff member combin consult merger acquisit uniqu featur first annapoli afford consult opportun develop varieti skill set furthermor empow consult take much project respon skill allow advanc talent individu base capabl success work conduct project team en

In [75]:
#### creating a set of multiworded skills and single worded skill 
skill_words = []
skill_keywords1 = []
skill_keywords2 = []
for i in skillset:
        skill_words = i.split(' ')
        if len(skill_words) > 1:
            skill_keywords2.append(i)
        else:
            skill_keywords1.append(i)

In [76]:
degree_dict = {'bs': 1, 'bachelor': 1, 'undergraduate': 1, 
               'master': 2, 'graduate': 2, 'mba': 2.5, 
               'phd': 3, 'ph.d': 3, 'ba': 1, 'ma': 2,
               'postdoctoral': 4, 'postdoc': 4, 'doctorate': 3}


degree_dict2 = {'advanced degree': 2, 'ms or': 2, 'ms degree': 2, '4 year degree': 1, 'bs/': 1, 'ba/': 1,
                '4-year degree': 1, 'b.s.': 1, 'm.s.': 2, 'm.s': 2, 'b.s': 1, 'phd/': 3, 'ph.d.': 3, 'ms/': 2,
                'm.s/': 2, 'm.s./': 2, 'msc/': 2, 'master/': 2, 'master\'s/': 2, 'bachelor\s/': 1}
degree_keywords2 = set(degree_dict2.keys())

In [78]:
import pandas as pd
df = pd.DataFrame()
df['Job Description'] = data
df['Year'] = jobs['year']
df['Function'] = jobs['Job_Function']
print(df)

                                         Job Description  ...                             Function
0       bdo summer leadership program pathway success...  ...                                Other
1       specialist dedic work client team solv client...  ...  Marketing/Business/Sales Management
2       first annapoli consult inc offer career oppor...  ...                                Other
3       primari respon manag compani one hundr thirti...  ...  Marketing/Business/Sales Management
4       epic academi earli work startup organ incr im...  ...                   Financial Services
...                                                  ...  ...                                  ...
46155   us aspen institut global nonprofit organ comm...  ...  Marketing/Business/Sales Management
46156   bachelor degree5 year experi product program ...  ...                                Other
46157   plan lead varieti duti relat gener ledger acc...  ...                                Other
46158   fs

In [82]:
# Initialize and train the model (this will take some time)
from gensim.models import word2vec
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# Initialize the "CountVectorizer" object, which is scikit-learn's bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",
                             tokenizer = None, 
                             preprocessor = None,
                             stop_words = None,  
                             max_features = 5000)


In [102]:
train_data_features = vectorizer.fit_transform(data)
print(train_data_features)

  (0, 4359)	2
  (0, 2586)	4
  (0, 3525)	4
  (0, 3257)	2
  (0, 4348)	2
  (0, 3108)	3
  (0, 72)	7
  (0, 4323)	4
  (0, 90)	3
  (0, 1048)	2
  (0, 3147)	2
  (0, 1660)	2
  (0, 4768)	1
  (0, 4558)	2
  (0, 3015)	3
  (0, 682)	1
  (0, 3573)	4
  (0, 2529)	2
  (0, 2743)	1
  (0, 1931)	1
  (0, 1417)	1
  (0, 4422)	1
  (0, 1241)	2
  (0, 2125)	1
  (0, 2723)	2
  :	:
  (46159, 3822)	1
  (46159, 2268)	1
  (46159, 2067)	1
  (46159, 2164)	1
  (46159, 1225)	2
  (46159, 628)	1
  (46159, 130)	2
  (46159, 405)	1
  (46159, 4352)	1
  (46159, 3425)	1
  (46159, 322)	1
  (46159, 3346)	1
  (46159, 1832)	3
  (46159, 3439)	1
  (46159, 1684)	1
  (46159, 3219)	1
  (46159, 1831)	1
  (46159, 4016)	1
  (46159, 526)	1
  (46159, 1448)	1
  (46159, 2648)	1
  (46159, 2514)	1
  (46159, 4639)	1
  (46159, 3349)	1
  (46159, 2353)	1


In [103]:
train_data_features = train_data_features.toarray()
# Numpy arrays are easy to work with, so convert the result to an array
#train_data_features = train_data_features.toarray()
train_data_features

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [104]:
train_data_features.shape

(46160, 5000)

In [105]:
# Take a look at the words in the vocabulary
vocab = vectorizer.get_feature_names()
print(vocab)

['00am', '00pm', '0in', '0px', '100k', '10am', '10k', '12th', '13th', '14th', '15th', '18th', '19th', '1b', '1st', '1yp', '20th', '21st', '2nd', '30am', '30th', '3d', '3pl', '3rd', '3yp', '401k', '4g', '4p', '4th', '59pm', '5pm', '5th', '6th', '8th', '9th', 'aa', 'aaa', 'aap', 'aarp', 'ab', 'abi', 'abid', 'abil', 'abinbev', 'abl', 'abreast', 'abroad', 'absenc', 'absolut', 'absorb', 'abstract', 'abt', 'abu', 'abund', 'ac', 'aca', 'academ', 'academi', 'academia', 'academicsinasia', 'accel', 'accentur', 'accept', 'access', 'accessori', 'accid', 'accolad', 'accommod', 'accompani', 'accomplish', 'accord', 'accordingli', 'account', 'accounting', 'accredit', 'accru', 'accrual', 'accumul', 'accur', 'accuraci', 'ace', 'acef', 'achiev', 'acknowledg', 'acli', 'acquaint', 'acquir', 'acquisit', 'acr', 'acrobat', 'across', 'act', 'action', 'activ', 'activities', 'actual', 'actuari', 'acumen', 'acut', 'ad', 'ada', 'adaaa', 'adapt', 'add', 'addit', 'address', 'adept', 'adequ', 'adequaci', 'adher', 'ad

In [107]:
# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

In [115]:
model = word2vec.Word2Vec(sentences=vocab, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

2020-09-10 04:57:06,999 : INFO : collecting all words and their counts
2020-09-10 04:57:07,000 : WARNING : Each 'sentences' item should be a list of words (usually unicode strings). First item here is instead plain <class 'str'>.
2020-09-10 04:57:07,001 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-09-10 04:57:07,019 : INFO : collected 36 word types from a corpus of 29828 raw words and 5000 sentences
2020-09-10 04:57:07,020 : INFO : Loading a fresh vocabulary
2020-09-10 04:57:07,022 : INFO : effective_min_count=40 retains 26 unique words (72% of original 36, drops 10)
2020-09-10 04:57:07,023 : INFO : effective_min_count=40 leaves 29751 word corpus (99% of original 29828, drops 77)
2020-09-10 04:57:07,024 : INFO : deleting the raw counts dictionary of 36 items
2020-09-10 04:57:07,025 : INFO : sample=0.001 downsamples 24 most-common words
2020-09-10 04:57:07,027 : INFO : downsampling leaves estimated 5098 word corpus (17.1% of prior 29751)
2020-09-10 04:

In [120]:
model.wv.most_similar('citizen')

KeyError: ignored

In [ ]:
from nltk import pos_tag
from nltk.stem import PorterStemmer

ps = PorterStemmer()


# process the job description.
def prepare_job_desc(desc):
    # tokenize description.
    tokens = word_tokenize(desc)
        
    # Parts of speech (POS) tag tokens.
    token_tag = pos_tag(tokens)
    
    # Only include some of the POS tags.
    include_tags = ['VBN', 'VBD', 'JJ', 'JJS', 'JJR', 'CD', 'NN', 'NNS', 'NNP', 'NNPS']
    filtered_tokens = [tok for tok, tag in token_tag if tag in include_tags]
    
    # stem words.
    stemmed_tokens = [tok.lower() for tok in filtered_tokens]
    return set(stemmed_tokens)

df['job_description_word_set'] = df['Job Description'].map(prepare_job_desc)


skill_keywords1_set = set([ps.stem(tok) for tok in skill_keywords1])# stem the keywords (since the job description is also stemmed.)
skill_keywords1_dict = {ps.stem(tok):tok for tok in skill_keywords1}# use this dictionary to revert the stemmed words back to the original.

degree_keywords1_set = set([ps.stem(tok) for tok in degree_dict.keys()])
degree_keywords1_dict = {ps.stem(tok):tok for tok in degree_dict.keys()}


In [ ]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
func_list = jobs['Job_Function'].unique()
func_list

array(['Other', 'Marketing/Business/Sales Management',
       'Financial Services', 'Professional Services', 'Technology',
       'Retail', 'Consulting', 'Engineering Services',
       'Real Estate & Construction', 'Healthcare & Pharmaceuticals',
       'Manufacturing', 'Multimedia Production / Graphic Design',
       'Transportation & Logistics', 'Other - Government Affairs'],
      dtype=object)

In [ ]:
skill_list = []
degree_list = []
skill_year = []
function = []
function_deg = []
tot_postings = []
postings_degree = []
degree_year = []
Job_Function = pd.DataFrame()

for func in func_list:
    Job_Function = df[df['Function'] == func] # Filtering based on job function
    for z in range(2011,2021):
        msk = Job_Function['Year'] == z # Filtering based on year

        num_postings = len(Job_Function[msk].index)
        for i in range(num_postings):
            job_desc = Job_Function[msk].iloc[i]['Job Description'].lower()
            job_desc_set = Job_Function[msk].iloc[i]['job_description_word_set']

            # check if the keywords are in the job description. Look for exact match by token.
            skill_words = skill_keywords1_set.intersection(job_desc_set)
            degree_words = degree_keywords1_set.intersection(job_desc_set)

            # check if longer keywords (more than one word) are in the job description. Match by substring.
            k = 0
            for skill_keyword2 in skill_keywords2:
                # skill keywords.
                if skill_keyword2 in job_desc:
                    skill_list.append(skill_keyword2)
                    k += 1

            # search for the minimum education.
            min_education_level = 999
            for degree_word in degree_words:
                level = degree_dict[degree_keywords1_dict[degree_word]]
                min_education_level = min(min_education_level, level)

            for degree_keyword2 in degree_keywords2:
                # longer keywords. Match by substring.
                if degree_keyword2 in job_desc:
                    level = degree_dict2[degree_keyword2]
                    min_education_level = min(min_education_level, level)

            # If none of the keywords were found, but the word degree is present, then assume it's a bachelors level.
            if min_education_level > 500:
                if 'degree' in job_desc:
                    min_education_level = 1

            skill_list += list(skill_words)
            degree_list.append(min_education_level)
        A = len(skill_list)
        B = len(skill_year)
        C = len(degree_list)
        D = len(degree_year)
        skill_year.extend([z] * (A-B))
        degree_year.extend([z] * (C-D))
        tot_postings.extend([num_postings] * (A-B))
        postings_degree.extend([num_postings] * (C-D))
    A = len(skill_year)
    B = len(function)
    C = len(degree_year)
    D = len(function_deg)
    function.extend([func] * (A-B))
    function_deg.extend([func] * (C-D))

In [ ]:
# create the list of skills/knowledge.
df_skills = pd.DataFrame(data={'cnt': skill_list, 'Year': skill_year, 'Job Function': function, 'Postingcount': tot_postings})
df_skills = df_skills.replace(skill_keywords1_dict)

# group some of the categories together.

msk = df_skills['cnt'] == 'ms excel'
df_skills.loc[msk, 'cnt'] = 'excel'

msk = df_skills['cnt'] == 'excel skills'
df_skills.loc[msk, 'cnt'] = 'excel'

msk = df_skills['cnt'] == 'strong excel'
df_skills.loc[msk, 'cnt'] = 'excel'

msk = df_skills['cnt'] == 'power bi'
df_skills.loc[msk, 'cnt'] = 'powerbi'

msk = df_skills['cnt'] == 'erp system'
df_skills.loc[msk, 'cnt'] = 'erp'

msk = df_skills['cnt'] == 'sql server'
df_skills.loc[msk, 'cnt'] = 'sql'

msk = df_skills['cnt'] == 'sqlexperience'
df_skills.loc[msk, 'cnt'] = 'sql'

msk = df_skills['cnt'] == 'sta'
df_skills.loc[msk, 'cnt'] = 'stata'

msk = df_skills['cnt'] == 'microsoft excel'
df_skills.loc[msk, 'cnt'] = 'excel'

msk = df_skills['cnt'] == 'ms powerpoint'
df_skills.loc[msk, 'cnt'] = 'powerpoint'

msk = df_skills['cnt'] == 'power point'
df_skills.loc[msk, 'cnt'] = 'powerpoint'

msk = df_skills['cnt'] == 'ms power point'
df_skills.loc[msk, 'cnt'] = 'powerpoint'

msk = df_skills['cnt'] == 'microsoft power point'
df_skills.loc[msk, 'cnt'] = 'powerpoint'

msk = df_skills['cnt'] == 'microsoft powerpoint'
df_skills.loc[msk, 'cnt'] = 'powerpoint'

msk = df_skills['cnt'] == 'six sigma'
df_skills.loc[msk, 'cnt'] = 'lean six sigma'

msk = df_skills['cnt'] == 'nlp'
df_skills.loc[msk, 'cnt'] = 'natural language processing'

msk = df_skills['cnt'] == 'convolutional'
df_skills.loc[msk, 'cnt'] = 'convolutional neural network'

msk = df_skills['cnt'] == 'cnn'
df_skills.loc[msk, 'cnt'] = 'convolutional neural network'

msk = df_skills['cnt'] == 'recurrent'
df_skills.loc[msk, 'cnt'] = 'recurrent neural network'

msk = df_skills['cnt'] == 'rnn'
df_skills.loc[msk, 'cnt'] = 'recurrent neural network'

msk = df_skills['cnt'] == 'knn'
df_skills.loc[msk, 'cnt'] = 'nearest neighbors'

msk = df_skills['cnt'] == 'svm'
df_skills.loc[msk, 'cnt'] = 'support vector machine'

msk = df_skills['cnt'] == 'machine vision'
df_skills.loc[msk, 'cnt'] = 'computer vision'

msk = df_skills['cnt'] == 'ab testing'
df_skills.loc[msk, 'cnt'] = 'a/b testing'

msk = df_skills['cnt'] == 'h20'
df_skills.loc[msk, 'cnt'] = 'h2o'

msk = df_skills['cnt'] == 'aws'
df_skills.loc[msk, 'cnt'] = 'amazon web services'

msk = df_skills['cnt'] == 'gcp'
df_skills.loc[msk, 'cnt'] = 'google cloud'

msk = df_skills['cnt'] == 'github'
df_skills.loc[msk, 'cnt'] = 'git'

msk = df_skills['cnt'] == 'postgressql'
df_skills.loc[msk, 'cnt'] = 'postgres'

msk = df_skills['cnt'] == 'tensor'
df_skills.loc[msk, 'cnt'] = 'tensorflow'

msk = df_skills['cnt'] == 'abi'
df_skills.loc[msk, 'cnt'] = 'application binary interface'


In [ ]:
df_skillcount = pd.DataFrame()
df_skills_top = pd.DataFrame()
for func in func_list:
    df_skillcount_f = df_skills[df_skills['Job Function'] == func]
    for k in range(2011, 2021):
        if k in df_skillcount_f['Year'].values:
            value = df_skillcount_f[df_skillcount_f['Year'] == k]['Postingcount'].iloc[0]
            df_skillcount = df_skillcount_f[df_skillcount_f['Year'] == k]['cnt'].value_counts().reset_index().rename(columns={'index': 'skill'}).iloc[:30]
            df_skillcount['Year'] = k
            df_skillcount['cnt'] = round((df_skillcount['cnt']/value) * 100)
            df_skillcount['Job Function'] = func
            df_skills_top = df_skills_top.append(df_skillcount)

In [ ]:
df_skills_top = df_skills_top.sort_values(['Job Function', 'Year', 'cnt'], ascending=[True, True, False])
print(df_skills_top)

              skill   cnt  Year                Job Function
0             excel  37.0  2011                  Consulting
1              word  11.0  2011                  Consulting
2        statistics  10.0  2011                  Consulting
3            access   9.0  2011                  Consulting
4             visio   8.0  2011                  Consulting
..              ...   ...   ...                         ...
25          powerbi   3.0  2020  Transportation & Logistics
26  team leadership   3.0  2020  Transportation & Logistics
27              sql   3.0  2020  Transportation & Logistics
28           python   3.0  2020  Transportation & Logistics
29              jsp   3.0  2020  Transportation & Logistics

[3439 rows x 4 columns]


In [ ]:
df_skills_top["Year"] = df_skills_top["Year"].astype(str)

import plotly.express as px

fig = px.bar(df_skills_top, x="skill", y="cnt", animation_frame ='Year', barmode = 'group')
fig.update_layout(barmode='group', xaxis_tickangle=45)
fig.show()

In [ ]:
df_skills_top['On Campus'] = 'NonOCI'

In [ ]:
df_skills_top.to_csv("NonOciSkills.csv")

In [ ]:
# create the list of degree.
df_degrees = pd.DataFrame(data={'cnt': degree_list, 
                                'Year': degree_year, 
                                'Job Function': function_deg, 
                                'Postingcount': postings_degree})
df_degrees['degree_type'] = ''


msk = df_degrees['cnt'] == 1
df_degrees.loc[msk, 'degree_type'] = 'bachelors'

msk = df_degrees['cnt'] == 2
df_degrees.loc[msk, 'degree_type'] = 'masters'

msk = df_degrees['cnt'] == 3
df_degrees.loc[msk, 'degree_type'] = 'phd'

msk = df_degrees['cnt'] == 4
df_degrees.loc[msk, 'degree_type'] = 'postdoc'

msk = df_degrees['cnt'] == 2.5
df_degrees.loc[msk, 'degree_type'] = 'mba'

msk = df_degrees['cnt'] > 500
df_degrees.loc[msk, 'degree_type'] = 'not specified'


df_degree_cnt = df_degrees['degree_type'].value_counts().reset_index().rename(columns={'index': 'degree'}).iloc[:50]


In [ ]:
DegreeData = pd.DataFrame()
for func in func_list:
    df_degreecount = df_degrees[df_degrees['Job Function'] == func]
    for k in range(2011, 2021):
        if k in df_degreecount['Year'].values:
            value = df_degreecount[df_degreecount['Year'] == k]['Postingcount'].iloc[0]
            df_MinDegree = df_degreecount[df_degreecount['Year'] == k]['degree_type'].value_counts().reset_index().rename(columns={'index': 'Degree'}).iloc[:30]
            df_MinDegree['Year'] = k
            df_MinDegree['degree_type'] = round((df_MinDegree['degree_type']/value) * 100)
            df_MinDegree['Job Function'] = func
            DegreeData = DegreeData.append(df_MinDegree)

In [ ]:
# visualize the degrees.

DegreeData["Year"] = DegreeData["Year"].astype(str)

import plotly.express as px

fig = px.bar(DegreeData, x="Degree", y="degree_type",color = 'Job Function', animation_frame ='Year', barmode = 'group')
fig.update_layout(barmode='group', xaxis_tickangle=45)
fig.show()



In [ ]:
DegreeData['On Campus'] = 'NonOCI'

In [ ]:
DegreeData.to_csv("NonOciDegree.csv")

In [1]:
pip install gensim

In [2]:
import warnings   
warnings.filterwarnings(action = 'ignore') 
  
import gensim 
from gensim.models import Word2Vec 


In [6]:
df_skills_top = df_skills_top.sort_values(['Job Function', 'Year', 'cnt'], ascending=[True, True, False])
print(df_skills_top)

NameError: ignored